In [4]:
import psycopg2
import os
import csv
import time
import numpy as np

from category_encoders import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GRU, Dropout, Input

In [ ]:
conn = psycopg2.connect(
  host='localhost',
  database='kkangjun',
  user='kkangjun',
  password=''
)

cur = conn.cursor()

In [ ]:
cur.execute('SELECT * FROM train_cong')

time.sleep(1)

data = cur.fetchall()
feature = next(data)

x_train = np.array([row[:-1] for row in data])
y_train = np.array([row[-1] for row in data])

In [ ]:
encoder = OrdinalEncoder()
x_train_enc = encoder.fit_transform(x_train)

scaler = MinMaxScaler()
x_train_scl = scaler.fit_transform(x_train_enc)

In [ ]:
def gru_model(layers=0):
  inputs = Input(shape=(5,))
  layer = GRU(64, activation='tanh')(inputs)
  layer = Dropout(0.2)(layer)
  
  for _ in range(layers):
    layer = GRU(64, activation='tanh')(inputs)
    layer = Dropout(0.2)(layer)
  
  outputs = Dense(1, name='output')(layer)
  
  model = Model(inputs=inputs, outputs=outputs)
  
  model.compile(optimizer='adam', loss='mean_squared_error')
  
  return model

In [ ]:
model = gru_model(3)

model.fit(x_train_scl, y_train, batch_size=128, epochs=10)